# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = 'KJ7432.9.A.QJT86'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PASS", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 2, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1S', 'insta_score': 0.755, 'expected_score': 70, 'adjustment': 38},
 {'call': '2S', 'insta_score': 0.241, 'expected_score': 31, 'adjustment': 12}]

In [4]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.944}]

In [5]:
bid.samples

[]

In [6]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D", "PASS", "3N" , "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.999}, {'call': 'X', 'insta_score': 0.0}]

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

KJxxxx.9.A.QJT8x Q98.ATx.K9x.xxxx T.KJxx.Qxxx.AK9x Axx.Q8xxx.JT8xx.
KJxxxx.9.A.QJT8x Q98.A8xx.Txxx.9x T.KTxx.KQJ8x.Axx Axx.QJxx.9xx.Kxx
KJxxxx.9.A.QJT8x A8x.JTxx.9xx.A9x 9.AQ8x.QJ8xx.Kxx QTx.Kxxx.KTxx.xx
KJxxxx.9.A.QJT8x Q8x.Qxx.xxxx.AKx A.KTxx.KQ98.9xxx T9x.AJ8xx.JTxx.x
KJxxxx.9.A.QJT8x Tx.xxxx.K9xx.9xx x.AKJx.J8xxx.AKx AQ98.QT8x.QTx.xx
KJxxxx.9.A.QJT8x A98.K8x.9xx.Kxxx x.QJxx.KJTxx.A9x QTx.ATxxx.Q8xx.x
KJxxxx.9.A.QJT8x AQT.Q8xxx.8xx.xx .Kxxx.KQTxxx.K9x 98xx.AJT.J9x.Axx
KJxxxx.9.A.QJT8x QTx.KQxx.98xx.xx x.AJT8.KQJxx.9xx A98.xxxx.Txx.AKx
KJxxxx.9.A.QJT8x A98.Txxx.xxx.9xx T.AJ8x.KQJ8x.AKx Qxx.KQxx.T9xx.xx
KJxxxx.9.A.QJT8x 98x.A8xx.KQJx.Kx T.KQJx.98xx.A9xx AQx.Txxx.Txxx.xx
KJxxxx.9.A.QJT8x 9x.QTxxx.9xx.Kxx Q.AK8x.KJxx.A9xx AT8x.Jxx.QT8xx.x
KJxxxx.9.A.QJT8x QT8.AQxx.8xxx.9x x.KT8x.KT9xx.Axx A9x.Jxxx.QJx.Kxx
KJxxxx.9.A.QJT8x Tx.AQJTx.JT9x.Kx A.K8xx.KQ8xx.xxx Q98x.xxx.xxx.A9x
KJxxxx.9.A.QJT8x AT9.T8x.KQ9x.K9x x.AQxx.Txxxx.Axx Q8x.KJxxx.J8x.xx
